In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

# First, we create a new instance of the Firefox driver
driver = webdriver.Firefox()

# List of URL endings for each subpage
url_endings = ['segunda-vuelta-de-elección-presidencial-2021-presidencial_candidatos-y-resultados_89S3sk+pJI0=Sk','elecciones-generales-2021-presidencial_candidatos-y-resultados_y0NC1IW14uw=NI', 'segunda-vuelta-de-elección-presidencial-2016-presidencial_candidatos-y-resultados_7p675u8oH7Q=6u', 'elecciones-generales-2016-presidencial_candidatos-y-resultados_LY8odlbsty4=8l','segunda-vuelta-de-la-elección-presidencial-2011-presidencial_candidatos-y-resultados_0xj1o+1NixM=j+','elecciones-generales-2011-presidencial_candidatos-y-resultados_24JPDKD@FrU=JK']

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL ending
for url_ending in url_endings:
    # Go to the website
    driver.get(f"https://infogob.jne.gob.pe/Eleccion/FichaEleccion/{url_ending}")

    # Add a delay so the elements can be found
    time.sleep(35)

    # Get the HTML of the page and pass it to BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find the table and the input element
    table = soup.find('table', {'id': 'gridEleccionResultadosElectorales'})
    input_element = soup.find('input', {'id': 'nombreEleccion'})

    # Get the value of the input element
    year = input_element['value']

    # Find all rows in the table
    rows = table.find_all('tr')

    # Create a list to store the data
    data = []

    # Loop through each row
    for row in rows:
        # Find all columns in each row
        cols = row.find_all('td')
        
        # Get the text from each column
        cols = [col.text.strip() for col in cols]
        
        # Add the columns to the data array
        data.append(cols)

    # Create a list to store the extracted data
    extracted_data = []

    # Loop through each list in data
    for data_list in data:
        
        if len(data_list) >= 4:
            # Extract the first and fourth items and add them to extracted_data
            extracted_data.append([data_list[0], data_list[3]])

    # Convert the extracted data to a DataFrame
    temp_df = pd.DataFrame(extracted_data, columns=['Organización Política', 'Total de votos'])

    # Create a new DataFrame with the new column
    new_df = pd.DataFrame({'Elecciones': [year]*len(temp_df)})

    # Concatenate the new DataFrame with the original DataFrame
    temp_df = pd.concat([new_df, temp_df], axis=1)

    # Append the DataFrame to the list
    dfs.append(temp_df)

# Finally we close the browser
driver.quit()

# To export into excel sheet, we concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs)

# Specify the path where you want to store the Excel file
file_path = r'C:\Users\jesus\OneDrive\Documentos\GitHub\QLAB_Summer_Python\assignments\assignment_5\group_2_ass_5_2024.xlsx'

# Use the 'to_excel' function to export the DataFrame
df.to_excel(file_path, index=False)




